## Query And Build

The purpose of this notebook is to query various tables from our SQLite database (constructed using create_db, populate_db, and main files), construct recidivism flags, and perform other cleaning functions. The build_all() function is run twice - once giving a dataset with recidivism labels corresponding to anyone reincarcerated within 1 year, the second time corresponding to anyone reincarcerated within 3 years.

The resulting datasets are saved as csv's and used in the notebook titled 'analysis' where we construct features, pre-process data, and run our models

In [1]:
import sqlite3
from sqlite3 import Error
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd
import config
import build_dataset as build

import importlib
import datetime
import re
import numpy as np
import pandas as pd



Bad key "text.kerning_factor" on line 4 in
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import importlib

In [5]:
importlib.reload(build)
importlib.reload(config)

<module 'config' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/config.py'>

In [6]:
data_final_1 = build.build_all(config.database_name,num_years=1)

Time Elapsed: 0:02:14.745235
Time Elapsed: 0:02:48.956136
891122
Dataset B # observations: 823722
% missing most serious offense: 0.03883602640442347
Total number of observations in dataset A + B:  903182
Cleaning dates and dropping missing
Total number of observations in dataset A + B:  888121
% still missing most serious offense: 0.02623966779301469
Drop observations missing most serious offense code
Total number of observations in dataset A + B:  864817
Querying database to get nextPrefix, nextOffense
Time Elapsed: 0:01:12.378648
Fix Dates
Get recidivism flag
Disc Infractions (362962, 3)
Hold out active sentences
Size of active sentences dataset:  32801
Drop observations with no recidivism flag (this will also drop active sentences, but we've already separated those)
Additional observations dropped are mostly of those who died in prison and therefore wont have a recidivate flag
Size of remaining dataset:  827731
Merging on our coded categories
% missing decided category 0.0077525186

In [7]:
data_final_3 = build.build_all(config.database_name,num_years=3)

Time Elapsed: 0:02:15.390164
Time Elapsed: 0:01:57.145201
891122
Dataset B # observations: 823722
% missing most serious offense: 0.03883602640442347
Total number of observations in dataset A + B:  903182
Cleaning dates and dropping missing
Total number of observations in dataset A + B:  888121
% still missing most serious offense: 0.02623966779301469
Drop observations missing most serious offense code
Total number of observations in dataset A + B:  864817
Querying database to get nextPrefix, nextOffense
Time Elapsed: 0:01:11.041770
Fix Dates
Get recidivism flag
Disc Infractions (362962, 3)
Hold out active sentences
Size of active sentences dataset:  32801
Drop observations with no recidivism flag (this will also drop active sentences, but we've already separated those)
Additional observations dropped are mostly of those who died in prison and therefore wont have a recidivate flag
Size of remaining dataset:  827731
Merging on our coded categories
% missing decided category 0.0140226716

In [90]:
data_final_1.head(2)

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,PRIMARY_OFFENSE_CODE,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,INFRACTION_PER_SENT,"(Count, FELON)","(Count, MISD.)",Primary offense code_x,Description (if needed)_x,Recidivate_Risk_Level,Needed a check?_x,Recidivate_Risk_Level_Lenient,Recidivate_Risk_Level_Harsh,Primary offense code_y,Description (if needed)_y,Current_Offense_Risk_Level,Needed a check?_y,Current_Offense_Risk_Level_Lenient,Current_Offense_Risk_Level_Harsh
0,0000004,AA,1983-07-12,SELL SCHEDULE II,EXPIRED,1984-07-11,1984-07-11,INACTIVE,INACTIVE,1984-07-11,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1961-10-15,ALASKA,UNKNOWN,BORN IN U.S.,SELL SCHEDULE II,NONE,NaT,0,0,NaN,0.0,0.0,2.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,SELL SCHEDULE II,Selling drugs on Schedule II - high potential ...,3.0,YES,2.0,4.0
1,0000006,AA,1973-01-30,WORTHLESS CHECK,EXPIRED,1973-03-28,0001-01-01,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,WORTHLESS CHECK,AB,1973-04-11,WORTHLESS CHECK,1973,0.0,1.0,0.0,0.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0
